In [1]:
from taxcalc import *
import pandas as pd
import numpy as np

In [2]:
nydata_temp = pd.read_csv('puf_ny_2017.csv')
nydata_temp['E00200p'] = nydata_temp['E00200']
nydata_temp['E00900p'] = nydata_temp['E00900']
nydata_temp['E02100p'] = nydata_temp['E02100']
e00600_temp = nydata_temp['E00600']
e00650 = nydata_temp['E00650']
e00600 = np.where(e00650 > e00600_temp, e00650, e00600_temp)
nydata_temp.drop(columns='E00600')
nydata_temp['E00600'] = e00600
nydata_temp.columns = [x.lower() for x in nydata_temp.columns]
nydata = nydata_temp.rename(columns={'recid': 'RECID', 'dsi': 'DSI', 'mars': 'MARS', 'midr': 'MIDR', 'xtot': 'XTOT'})

In [3]:
ny_weights = pd.DataFrame()
ny_weights['WT2017'] = nydata['weight_state'] * 100

# ny_weights['WT2018'] = ny_weights['WT2017'] * 1.01
# ny_weights['WT2019'] = ny_weights['WT2018'] * 1.01
# ny_weights['WT2020'] = ny_weights['WT2019'] * 1.01
# ny_weights['WT2021'] = ny_weights['WT2020'] * 1.01
# ny_weights['WT2022'] = ny_weights['WT2021'] * 1.01
# ny_weights['WT2023'] = ny_weights['WT2022'] * 1.01
# ny_weights['WT2024'] = ny_weights['WT2023'] * 1.01
# ny_weights['WT2025'] = ny_weights['WT2024'] * 1.01
# ny_weights['WT2026'] = ny_weights['WT2025'] * 1.01
# ny_weights['WT2027'] = ny_weights['WT2026'] * 1.01
# ny_weights['WT2028'] = ny_weights['WT2027'] * 1.01
# ny_weights['WT2029'] = ny_weights['WT2028'] * 1.01

In [4]:
salt_ref = {"ID_AllTaxes_c": {2017: [10000, 10000, 5000, 10000, 10000]}}

nyrec = Records(data=nydata, start_year=2017, weights=ny_weights)
usrec = Records()
            
pol = Policy()
pol_ref = Policy()
pol_ref.implement_reform(salt_ref)
            
calc_base_ny = Calculator(records=nyrec, policy=pol)
calc_ref_ny = Calculator(records=nyrec, policy=pol_ref)
            
calc_base_us = Calculator(records=usrec, policy=pol)
calc_ref_us = Calculator(records=usrec, policy=pol_ref)

In [5]:
calc_base_ny.advance_to_year(2017)
calc_base_ny.calc_all()

calc_ref_ny.advance_to_year(2017)
calc_ref_ny.calc_all()

calc_base_us.advance_to_year(2017)
calc_base_us.calc_all()

calc_ref_us.advance_to_year(2017)
calc_ref_us.calc_all()

In [6]:
dist_base_ny, dist_ref_ny = calc_base_ny.distribution_tables(calc_ref_ny, 'standard_income_bins')
dist_ny = pd.DataFrame()
dist_ny['funits_ny(#m)'] = dist_base_ny['count']
dist_ny['itax_ny_base($b)'] = dist_base_ny['iitax']
dist_ny['itax_ny_ref($b)'] = dist_ref_ny['iitax']
print('2017 Distributional Table for NY')
dist_ny

2017 Distributional Table for NY


,funits_ny(#m),itax_ny_base($b),itax_ny_ref($b)
<$0K,0.08,0.215,0.251
=$0K,0.02,-0.009,-0.009
$0-10K,1.02,-0.307,-0.306
$10-20K,1.20,-0.566,-0.565
$20-30K,1.06,0.078,0.079
$30-40K,0.94,0.907,0.910
$40-50K,0.82,1.682,1.692
$50-75K,1.47,6.319,6.400
$75-100K,0.90,7.129,7.276
$100-200K,1.44,22.627,23.664


In [7]:
dist_base_us, dist_ref_us = calc_base_us.distribution_tables(calc_ref_us, 'standard_income_bins')
dist_us = pd.DataFrame()
dist_us['funits_us(#m)'] = dist_base_us['count']
dist_us['itax_us_base($b)'] = dist_base_us['iitax']
dist_us['itax_us_ref($b)'] = dist_ref_us['iitax']
print('2017 Distributional Table for US')
dist_us

2017 Distributional Table for US


,funits_us(#m),itax_us_base($b),itax_us_ref($b)
<$0K,1.23,2.319,2.529
=$0K,3.33,-0.516,-0.516
$0-10K,21.43,-7.023,-7.018
$10-20K,24.51,-35.932,-35.930
$20-30K,19.04,-19.371,-19.361
$30-40K,15.51,1.428,1.450
$40-50K,13.31,18.444,18.490
$50-75K,23.02,83.968,84.257
$75-100K,13.81,97.817,98.470
$100-200K,23.87,347.954,354.768


In [10]:
diff_table_ny = calc_base_ny.difference_table(calc_ref_ny, 'standard_income_bins', 'iitax')
diff_ny = pd.DataFrame()
diff_ny['count_ny(#m)'] = diff_table_ny['count']
diff_ny['perc_inc_ny'] = diff_table_ny['perc_inc']
diff_ny['mean_ny'] = diff_table_ny['mean']
diff_ny['tot_change_ny($b)'] = diff_table_ny['tot_change']
print('2017 Difference Table for NY')
diff_ny

2017 Difference Table for NY


,count_ny(#m),perc_inc_ny,mean_ny,tot_change_ny($b)
<$0K,0.08,5.5,440.7,0.036
=$0K,0.02,0.0,0.0,0.000
$0-10K,1.02,0.1,0.7,0.001
$10-20K,1.20,0.0,1.1,0.001
$20-30K,1.06,0.2,0.8,0.001
$30-40K,0.94,0.6,3.6,0.003
$40-50K,0.82,1.8,12.4,0.010
$50-75K,1.47,5.8,54.9,0.081
$75-100K,0.90,14.3,162.5,0.146
$100-200K,1.44,44.7,720.6,1.037


In [11]:
diff_table_us = calc_base_us.difference_table(calc_ref_us, 'standard_income_bins', 'iitax')
diff_us = pd.DataFrame()
diff_us['count_us(#m)'] = diff_table_us['count']
diff_us['perc_inc_us'] = diff_table_us['perc_inc']
diff_us['mean_us'] = diff_table_us['mean']
diff_us['tot_change_us($b)'] = diff_table_us['tot_change']
print('2017 Difference Table for US')
diff_us

2017 Difference Table for US


,count_us(#m),perc_inc_us,mean_us,tot_change_us($b)
<$0K,1.23,3.0,169.9,0.210
=$0K,3.33,0.0,0.0,0.000
$0-10K,21.43,0.0,0.2,0.005
$10-20K,24.51,0.0,0.1,0.002
$20-30K,19.04,0.1,0.5,0.010
$30-40K,15.51,0.3,1.4,0.022
$40-50K,13.31,1.0,3.5,0.046
$50-75K,23.02,2.3,12.6,0.290
$75-100K,13.81,6.8,47.2,0.653
$100-200K,23.87,30.1,285.5,6.814
